In [1]:
import pandas as pd

import os
dataset_path = "/Users/bruce/Downloads/datasets/so/pastThreeMonth"

filenames = list(os.walk(dataset_path))[0][2]

Q_filenames = [name for name in filenames if name[0]=="Q"]

A_filenames = [name for name in filenames if name[0]=="A"]

### 读取并拆分数据

In [2]:
q_df = None
for i in range(len(Q_filenames)):
    qfile_path = os.path.join(dataset_path, Q_filenames[i])
    if i==0:
        q_df = pd.read_csv(qfile_path)
    else:
        tmp_df = pd.read_csv(qfile_path)
        q_df = pd.concat([q_df, tmp_df])
 

q_df.drop_duplicates(subset=['Id'], keep='first', inplace=True)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
a_df = None
for i in range(len(A_filenames)):
    afile_path = os.path.join(dataset_path, A_filenames[i])
    if i==0:
        a_df = pd.read_csv(afile_path)
    else:
        tmp_df = pd.read_csv(afile_path)
        a_df = pd.concat([a_df, tmp_df])

a_df.drop_duplicates(subset=['Id'], keep='first', inplace=True)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,12,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# 数据中前23列是post，23列之后是user
post_columns = a_df.columns[:23]
user_columns = a_df.columns[23:]

In [5]:
# 将question、answer、user分开
u_df = pd.concat([a_df[user_columns], q_df[user_columns]])
user_columns = [col for col in user_columns]
user_columns[0] = 'Id'
user_columns[2] = 'CreationDate'
u_df.columns = user_columns
a_df = a_df[post_columns]
q_df = q_df[post_columns]

### 1. 获取label

330350

In [7]:
q_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330350 entries, 0 to 41831
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     330350 non-null  int64  
 1   PostTypeId             330350 non-null  int64  
 2   AcceptedAnswerId       175232 non-null  float64
 3   ParentId               0 non-null       float64
 4   CreationDate           330350 non-null  object 
 5   DeletionDate           0 non-null       float64
 6   Score                  330350 non-null  int64  
 7   ViewCount              330350 non-null  int64  
 8   Body                   330350 non-null  object 
 9   OwnerUserId            330350 non-null  int64  
 10  OwnerDisplayName       110 non-null     object 
 11  LastEditorUserId       161412 non-null  float64
 12  LastEditorDisplayName  153 non-null     object 
 13  LastEditDate           161559 non-null  object 
 14  LastActivityDate       330350 non-nul

In [8]:
# 使用排序来会的分数越大，时间越早的答案
a_df['-Score'] = a_df['Score']*(-1)
sort_df = a_df.sort_values(by=['ParentId', '-Score', 'CreationDate']).loc[:,['ParentId','Score', 'CreationDate', 'Id']]

In [9]:
# 然后使用df去重，仅保留第一条
sort_df.drop_duplicates(subset=['ParentId'], keep='first', inplace=True)

In [10]:
sort_df.shape

(329751, 4)

In [13]:
a_df.drop(columns=['-Score'], inplace=True)

In [14]:
# 拼接
qtime_df = q_df.loc[:, ['Id', 'CreationDate']]
sort_df = pd.merge(left=sort_df, right=qtime_df, left_on='ParentId', right_on = 'Id')

In [15]:
# 设置列名
sort_df.columns = ['ParentId', 'a_Score', 'a_CreationDate', 'a_Id', 'q_Id', 'q_CreationDate']

In [16]:
import time
# 将字符串格式时间转换为时间戳
def str_to_timestep(s):
    timeArray = time.strptime(s.split('.')[0], "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp

In [17]:
sort_df['a_CreationDateStamp'] = sort_df['a_CreationDate'].apply(str_to_timestep)
sort_df['q_CreationDateStamp'] = sort_df['q_CreationDate'].apply(str_to_timestep)
# 计算回答和提问的相隔时间，单位是s
sort_df['TimeDelta'] = sort_df['a_CreationDateStamp'] - sort_df['q_CreationDateStamp']

#### <font color='blue'>获得了label，包含问题的id，答案的id， 时间间隔</font>

In [18]:
label_df = sort_df.loc[:, ['q_Id', 'a_Id', 'TimeDelta']]
del sort_df

In [19]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327349 entries, 0 to 327348
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   q_Id       327349 non-null  int64
 1   a_Id       327349 non-null  int64
 2   TimeDelta  327349 non-null  int64
dtypes: int64(3)
memory usage: 10.0 MB


### 2. 存储label、user、answer、question

In [ ]:
output_path = os.path.join(dataset_path, 'output')
output_filepath = os.path.join(output_path, 'label.csv')
label_df.to_csv(output_filepath)

In [ ]:
user_filepath = os.path.join(output_path, 'user.csv')
ans_filepath = os.path.join(output_path, 'answer.csv')
ques_filepath = os.path.join(output_path, 'question.csv')

u_df.to_csv(user_filepath)
a_df.to_csv(ans_filepath)
q_df.to_csv(ques_filepath)

### 3. 计算基于Tag的特征 

In [31]:
def handle_tag(tag):
    # 去除尖括号
    return tag.replace("<", " ").replace(">", " ").strip()

In [27]:
# 因为数据原因，部分数据没有label，因此将将没有label的数据剔除
q_df = label_df.merge(how="left", right = q_df, left_on='q_Id', right_on='Id')

q_df.drop(columns=['q_Id', 'a_Id', 'TimeDelta'], inplace=True)

In [32]:
# 处理tag
q_df["Tags"] = q_df["Tags"].apply(handle_tag)
q_df['Tags'] = q_df['Tags'].apply(lambda x: x.split())

In [33]:
# tag和question之间的关系
from tqdm import trange

tag_questions = []
q_num = q_df.shape[0]
for i in trange(q_num):
    q_id = q_df.iloc[i]['Id']
    tags = q_df.iloc[i]['Tags']
    for t in tags:
        tag_questions.append([t, q_id])

100%|██████████| 327349/327349 [02:06<00:00, 2585.84it/s]


#### <font color='blue'>(1)获得tag的流行度</font>

tag和question对应的dataframe

In [34]:
tag_ques_df = pd.DataFrame(tag_questions)

tag_ques_df.columns=['Tag', 'q_Id']

tag_features_df = tag_ques_df.groupby('Tag').size().reset_index(name='QuestionsNumber').sort_values(by=['Tag'])

跟label拼接，得到tag、question、time对应关系

In [35]:
tag_ques_time_df = pd.merge(left = tag_ques_df, right = label_df,left_on=tag_ques_df.q_Id, right_on=label_df.q_Id)

In [36]:
threshold = 60*60  # 一小时
tag_ques_time_df['WhetherQuick'] = tag_ques_time_df['TimeDelta']<threshold
tag_ques_time_df['WhetherQuick'] = tag_ques_time_df['WhetherQuick'].astype('float')

In [37]:
# 每个tag下平均响应时间
tag_time_df = tag_ques_time_df.groupby('Tag')['TimeDelta'].mean().reset_index(name='AverageTime'\
                                                                             ).sort_values(by=['Tag'])
# 每个tag下得到快速响应的问题个数
tag_quick_df = tag_ques_time_df.groupby('Tag')['WhetherQuick'].sum().reset_index(name='QuickNumber'\
                                                                                ).sort_values(by=['Tag'])

#### <font color='blue'>(2)获得tag的平均响应时间、快速个数</font>

In [38]:
tag_features_df['AverageTime'] = tag_time_df['AverageTime'].astype(int)
tag_features_df['QuickNumber'] = tag_quick_df['QuickNumber'].astype(int)

In [39]:
del tag_time_df;
del tag_quick_df;

一共有26883个tag，其中76个响应时间为Nan，可能是数据丢失的问题,分别使用0和中位数来填充

In [40]:
tag_features_df['QuickNumber']=tag_features_df['QuickNumber'].fillna(0)
tag_features_df['AverageTime']=tag_features_df['AverageTime'].fillna(tag_features_df['AverageTime'].median())

#### (3)存储tag具有的属性

In [ ]:
tag_features_filepath = os.path.join(output_path, 'tag_features.csv')
tag_features_df.to_csv(tag_features_filepath)

#### (4) 通过tag属性来计算question的tag-based属性 

In [42]:
tmp_df = pd.merge(left = tag_ques_df, right = tag_features_df, left_on = 'Tag', right_on = 'Tag')

In [43]:
tag_popularity = tmp_df.groupby('q_Id')['QuestionsNumber'\
                                       ].sum().reset_index(name='TagPopularity').sort_values(by=['q_Id'])

In [44]:
tag_averageTime = tmp_df.groupby('q_Id')['AverageTime'\
                                        ].mean().reset_index(name='TagAverageTime').sort_values(by=['q_Id'])

In [45]:
tag_averageQuick = tmp_df.groupby('q_Id')['QuickNumber'\
                                        ].mean().reset_index(name='TagAverageQuick').sort_values(by=['q_Id'])

In [46]:
tag_averageQuick.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327349 entries, 0 to 327348
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   q_Id             327349 non-null  int64  
 1   TagAverageQuick  327349 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 7.5 MB


In [47]:
tag_based_features = tag_popularity
tag_based_features['TagAverageTime'] = tag_averageTime['TagAverageTime']
tag_based_features['TagAverageQuick'] = tag_averageQuick['TagAverageQuick']

#### (5)存储tag-based的属性 

In [ ]:
tag_based_features_filepath = os.path.join(output_path, 'tag_based_features.csv')
tag_based_features.to_csv(tag_based_features_filepath)

### 3. 计算不基于Tag的特征, 即问题本身的特征

In [48]:
self_features = pd.DataFrame()

In [49]:
self_features['Id'] = q_df['Id']

In [50]:
self_features['num_code_snippet'] = 0
self_features['code_len'] = 0
self_features['num_image'] = 0
self_features['body_len'] = 0
self_features['title_len'] = 0
self_features['end_que_mark'] = 0
self_features['begin_que_word'] = 0
self_features['is_weekend'] = 0

#### (1) 通过问题的Body、Title、CreationDate来获得question自身的特征

In [51]:
import re
from datetime import datetime

code_pattern = re.compile(r"<code>(.*?)</code>", flags=re.DOTALL)
img_pattern = re.compile(r"<img(.*?)>", flags=re.DOTALL)

def get_num_code_snippet(s):
    res = code_pattern.findall(s)
    return len(res)

def get_code_len(s):
    res = code_pattern.findall(s)
    code_len = sum([len(code) for code in res])
    return code_len

def get_num_img(s):
    res = img_pattern.findall(s)
    return len(res)

def get_body_len(s):
    return len(s)

def get_title_len(s):
    return len(s)

def get_end_que_mark(s):
    s = s.strip()
    return len(s)!=0 and s[-1] == '?'

def get_begin_que_word(s):
    s = s.strip().lower()
    return len(s)!=0 and (s.startswith('wh') or s.startswith('how'))

def get_is_weekend(s):
    date_time_obj = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    return date_time_obj.weekday()>=5    # 周一是0，周二是1，... , 周日是6

In [52]:
from tqdm import tqdm

tqdm.pandas()

self_features['num_code_snippet'] = q_df['Body'].progress_apply(get_num_code_snippet)
self_features['code_len'] = q_df['Body'].progress_apply(get_code_len)
self_features['num_image'] = q_df['Body'].progress_apply(get_num_img)
self_features['body_len'] = q_df['Body'].progress_apply(get_body_len)
self_features['title_len'] = q_df['Title'].progress_apply(get_title_len)
self_features['end_que_mark'] = q_df['Title'].progress_apply(get_end_que_mark)
self_features['begin_que_word'] = q_df['Title'].progress_apply(get_begin_que_word)
self_features['is_weekend'] = q_df['CreationDate'].progress_apply(get_is_weekend)

/Users/bruce/opt/anaconda3/envs/py38_torch/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 327349/327349 [00:04<00:00, 67841.43it/s]


#### (2) 存储问题自身特征

In [ ]:
self_features_filepath = os.path.join(output_path, 'self_features.csv')
self_features.to_csv(self_features_filepath)

#### (3) 将问题self和tag based的特征合并起来进行存储

In [53]:
self_and_tag_features = pd.merge(left=self_features, right=tag_based_features, left_on='Id', right_on='q_Id')

In [54]:
self_and_tag_features.drop(columns=['q_Id'], inplace=True)

In [55]:
self_and_tag_features['end_que_mark'] = self_and_tag_features['end_que_mark'].astype(int)
self_and_tag_features['begin_que_word'] = self_and_tag_features['begin_que_word'].astype(int)
self_and_tag_features['is_weekend'] = self_and_tag_features['is_weekend'].astype(int)

In [58]:
features_df = self_and_tag_features
features_filepath = os.path.join(output_path, 'feature.csv')
features_df.to_csv(features_filepath)

## 最后提取得到features和label

In [63]:
(label_df["q_Id"].sort_values() == features_df["Id"].sort_values()).sum()

327349

In [64]:
features_df

,Id,num_code_snippet,code_len,num_image,body_len,title_len,end_que_mark,begin_que_word,is_weekend,TagPopularity,TagAverageTime,TagAverageQuick
0,63679634,1,688,0,1434,61,0,0,0,46348,36998.000000,10349.333333
1,63679639,2,2189,0,2738,48,0,0,0,28389,76474.250000,4026.750000
2,63679644,3,510,0,1088,50,0,0,0,243,159547.000000,54.000000
3,63679645,2,609,0,1034,77,0,1,0,4774,96742.000000,2825.000000
4,63679646,3,581,0,990,29,0,0,0,1081,99484.666667,134.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
327344,65082861,2,1369,0,2011,72,1,1,0,59369,73924.000000,12305.333333
327345,65082862,2,33,0,399,69,1,1,0,53554,101735.000000,11226.000000
327346,65083411,2,1247,0,1772,36,0,0,1,2580,147245.333333,360.333333
327347,65092147,1,1810,1,2872,62,1,1,0,65164,183770.750000,10890.250000


In [65]:
label_df

,q_Id,a_Id,TimeDelta
0,63679634,63679739,1047
1,63679639,63679836,1943
2,63679644,63680300,6848
3,63679645,63680052,4312
4,63679646,63680417,7919
...,...,...,...
327344,65082861,65083253,3099
327345,65082862,65083155,2316
327346,65083411,65083412,94328
327347,65092147,65092148,1515779


In [66]:
tag_features_df

,Tag,QuestionsNumber,AverageTime,QuickNumber
0,.a,1,273,1
1,.app,1,685,1
2,.class-file,1,4446,0
3,.d.ts,4,1123575,0
4,.doc,1,58014,0
...,...,...,...,...
26802,zxing,20,142339,4
26803,zxing-js,2,55545,0
26804,zxing.net,2,14762,0
26805,zynq,5,886832,0


In [ ]:
label_df.info()

In [ ]:
label_df.loc[label_df['TimeDelta'].isnull(), :]

In [ ]:
features_df.info()